In [50]:
import os
import random
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text

from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import SeparableConv1D
from tensorflow.python.keras.layers import MaxPooling1D
from tensorflow.python.keras.layers import GlobalAveragePooling1D

## Load dataset

I will use the [IMDb movie reviews sentiment analysis dataset](https://ai.stanford.edu/%7Eamaas/data/sentiment/) which contains **movie reviews** posted by people on the [IMDb website](https://www.imdb.com/), as well as the corresponding labels ("positive” or “negative”) indicating whether the reviewer liked the movie or not. There are 50,000 movie reviews divided into 25,000 reviews for training and 25,000 reviews for testing. The training and test sets are balanced, meaning they contain the same number of positive and negative reviews.

The data samples may be in a specific order. A simple best practice to ensure the model is not affected by data order is to always first shuffle the data.

In [10]:
def load_dataset(data_path, seed=123):

    """Loads the IMDb movie reviews sentiment analysis dataset.

    Arguments
    - data_path: string, path to the data directory.
    - seed: int, seed for randomizer.

    Returns
    - A tuple of training and validation data.
    - Number of training samples: 25000
    - Number of test samples: 25000
    - Number of categories: 2 (0 - negative, 1 - positive)

    References
    - Download and uncompress archive from:
    http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    """

    imdb_data_path = os.path.join(data_path, 'aclImdb')

    # Load the training data
    train_texts = []
    train_labels = []
    for category in ['pos', 'neg']:
        train_path = os.path.join(imdb_data_path, 'train', category)
        for fname in sorted(os.listdir(train_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(train_path, fname), encoding='utf-8') as f:
                    train_texts.append(f.read())
                train_labels.append(0 if category == 'neg' else 1)

    # Load the validation data
    test_texts = []
    test_labels = []
    for category in ['pos', 'neg']:
        test_path = os.path.join(imdb_data_path, 'test', category)
        for fname in sorted(os.listdir(test_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(test_path, fname), encoding='utf-8') as f:
                    test_texts.append(f.read())
                test_labels.append(0 if category == 'neg' else 1)

    # Shuffle the training data and labels
    random.seed(seed)
    random.shuffle(train_texts)
    random.seed(seed)
    random.shuffle(train_labels)

    return ((train_texts, np.array(train_labels)),
            (test_texts, np.array(test_labels)))

## Explore dataset

In [11]:
train_data, test_data = load_dataset(r"C:\Users\danie\Desktop\Projects\sentiment-analysis")

In [13]:
print(f"Number of training samples: {len(train_data[0])}")
print(f"Number of test samples: {len(test_data[0])}")

Number of training samples: 25000
Number of test samples: 25000


Let's see how our data looks like and check if the sentiment label corresponds to the sentiment of the review in a random sample:

In [27]:
train_data[0][5]

'A pointless movie with nothing but gratuitous violence. The only fun I had was playing "spot the location", as much of it was filmed in my home town of Regina, Saskatchewan. I like to support locally produced films but this one was a major disappointment.'

The expected sentiment (negative) matches the sample’s label:

In [28]:
train_data[1][5]

0

## Chose a model

[Google Developers](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) have created the following model selection algorithm and flowchart attempting to significantly simplify the process of selecting a text classification model. For a given dataset, our goal is to find the algorithm that achieves close to maximum accuracy while minimizing computation time required for training.

Models can be broadly classified into two categories:

- **N-gram models (Models that just see text as “bags” (sets) of words):** With n-gram vector representation, we discard a lot of information about word order and grammar. This representation is used in conjunction with models that don’t take ordering into account, such as logistic regression, simple multi-layer perceptrons (MLPs, or fully-connected neural networks), gradient boosted trees and support vector machines.

- **Sequence models (Models that use word ordering information):** For some text samples, word order is critical to the text’s meaning. Models such as convolutional neural networks (CNNs), and recurrent neural networks (RNNs) can infer meaning from the order of words in a sample.


The ratio of *number of samples* to *number of words per sample* correlates with which model performs better. 
- When the value for this ratio is **small (<1500)**, small multi-layer perceptrons that take **n-grams** as input perform better or at least as well as sequence models. MLPs are simple to define and understand, and they take much less compute time than sequence models. 
- When the value for this ratio is **large (>= 1500)**, a **sequence** model is a better option. Sequence models are better when there are a large number of small, dense vectors. This is because embedding relationships are learned in dense space, and this happens best over many samples.

<br>

**Model selection algorithm:**
1. Calculate the *number of samples* to *number of words per sample* ratio.
2. If this ratio is less than 1500, tokenize the text as n-grams and use a simple multi-layer perceptron (MLP) model to classify them (left branch in the flowchart below):
- Split the samples into word n-grams and convert the n-grams into vectors.
- Score the importance of the vectors and then select the top 20K using the scores.
- Build an MLP model.
3. If the ratio is greater than 1500, tokenize the text as sequences and use a sepCNN model to classify them (right branch in the flowchart below):
- Split the samples into words and select the top 20K words based on their frequency.
- Convert the samples into word sequence vectors.
- If the ratio is less than 15K, using a fine-tuned pre-trained embedding with the sepCNN model will likely provide the best results.
4. Measure the model performance with different hyperparameter values to find the best model configuration for the dataset.

<br>

**Flowchart:**

In the flowchart below, the yellow boxes indicate data and model preparation processes. Grey boxes and green boxes indicate choices they considered for each process. Green boxes indicate their recommended choice for each process.

<div style="width: 700px; overflow: hidden;">
    <img src="https://developers.google.com/static/machine-learning/guides/text-classification/images/TextClassificationFlowchart.png" width="100%" alt="Your Image">
</div>

Let's compute the *number of samples* to *number of words per sample* ratio:

In [29]:
def get_num_words_per_sample(sample_texts):
    """Gets the median number of words per sample given corpus.

    Arguments
    - sample_texts: list, sample texts.

    Returns
    - int, median number of words per sample.
    """
    num_words = [len(s.split()) for s in sample_texts]
    return np.median(num_words)

In [46]:
25000 / get_num_words_per_sample(train_data[0])

143.67816091954023

In the case of our IMDb review dataset, the ratio of *number of samples* to *number of words per sample* is less than 1500 so we should choose a n-gram model. We will create a n-gran model and sequence model for practice and comparison.

## Data pre-procesing

Machine learning algorithms take numbers as inputs. This means that we will need to convert the texts into numerical vectors. There are two steps to this process:

- **1. Tokenization:** Divide the texts into words or smaller sub-texts, which will enable good generalization of relationship between the texts and the labels. This determines the “vocabulary” of the dataset (set of unique tokens present in the data).
- **2. Vectorization:** Define a good numerical measure to characterize these texts.

Let’s see how to perform these two steps for both **n-gram vectors** and **sequence vectors**, as well as how to optimize the vector representations using feature selection and normalization techniques.

### N-gram vectors

In an n-gram vector, text is represented as a **collection of unique n-grams: groups of n adjacent tokens** (typically, words).

<br>

**1. Tokenization**

First, we have to split (tokenize) the text samles into word unigrams and bigrams. Thus, we will determines the "vocabulary" of the dataset. In the case of the text sample *'The mouse ran up the clock'*.

- The word unigrams (n = 1) are ['the', 'mouse', 'ran', 'up', 'clock']

- The word bigrams (n = 2) are ['the mouse', 'mouse ran', 'ran up', 'up the', 'the clock']

<br>

**2. Vectorization**

Once we have split our text samples into n-grams, we need to turn these n-grams into numerical vectors that our machine learning models can process. In the case of the text samples *'The mouse ran up the clock'* and *'The mouse ran down'*.

- The indexes assigned to the unigrams and bigrams would be {'clock': 0, 'down': 1, 'mouse': 2, 'mouse ran': 3, 'ran': 4, 'ran down': 5, 'ran up': 6, 'the': 7, 'the clock': 8, 'the mouse': 9, 'up': 10, 'up the': 11}


Once indexes are assigned to the n-grams, we typically vectorize the text samples using one-hot encoding, count encoding, or **Tf-idf encoding**. This last option is recommend for vectorizing n-grams. In the case of the  text sample *'The mouse ran up the clock'*.
- The vectorization using Tf-idf encoding would be [0.33, 0, 0.23, 0.23, 0.23, 0, 0.33, 0.47, 0.33, 0.23, 0.33, 0.33]

<br>

**3. Feature selection**

When we convert all of the texts in a dataset into word uni+bigram tokens, we may end up with tens of thousands of tokens. Not all of these tokens/features contribute to label prediction so we can drop certain tokens, for instance those that occur extremely rarely across the dataset. We can also measure feature importance (how much each token contributes to label predictions), and only include the most informative tokens. Two commonly used functions to calculate feature importance are **f_classif** and chi2. In addition, it has been noticed that accuracy peaks at around 20,000 features for many datasets.

<br>

The following code:
- Tokenize text samples into word unigrams + bigrams.
- Vectorize using tf-idf encoding.
- Select only the top 20,000 features from the vector of tokens by discarding tokens that appear fewer than 2 times and using f_classif to calculate feature importance.

Vectorization parameters:

In [ ]:
# Range (inclusive) of n-gram sizes for tokenizing text
NGRAM_RANGE = (1, 2)

# Whether text should be split into word or character n-grams ('word' or 'char')
TOKEN_MODE = 'word'

# Limit on the number of features. We use the top 20K features
TOP_K = 20000

# Minimum document/corpus frequency below which a token will be discarded
MIN_DOCUMENT_FREQUENCY = 2

In [48]:
def ngram_vectorize(train_texts, train_labels, val_texts):
    """Vectorizes texts as n-gram vectors
    
    1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams

    Arguments:
    - train_texts: list, training text strings
    - train_labels: np.ndarray, training labels
    - val_texts: list, validation text strings

    Returns:
    - x_train, x_val: vectorized training and validation texts
    """

    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
        'ngram_range': NGRAM_RANGE,
        'dtype': 'int32',
        'strip_accents': 'unicode',
        'decode_error': 'replace',
        'analyzer': TOKEN_MODE,
        'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)

    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

In [ ]:
train_data

### Sequence vectors

In a sequence vector, text is represented as a **sequence of tokens, preserving order**.

<br>

**1. Tokenization**

Text can be represented as either a sequence of characters, or a sequence of words. Using word-level representation provides better performance than character tokens. Using character tokens makes sense only if texts have lots of typos.

<br>

**2. Vectorization**

Once we have converted our text samples into sequences of words, we need to turn these sequences into numerical vectors. The example below shows the indexes assigned to the unigrams generated for two texts, and then the sequence of token indexes to which the first text is converted.

- Texts: 'The mouse ran up the clock' and 'The mouse ran down'
- Index assigned for every token: {'the': 1, 'mouse': 2, 'ran': 3, 'up': 4,'clock': 5, 'down': 6}.
- Sequence of token indexes: 'The mouse ran up the clock' = [1, 2, 3, 4, 1, 5]

Note that 'the' occurs most frequently, so the index value of 1 is assigned to it. Also some libraries reserve index 0 for unknown tokens, as is the case here.

To vectorize the token sequences we can use one-hot encoding, or **word embeddings**. This last option is recommend for vectorizing sequences since words have meaning(s) associated with them. As a result, we can represent word tokens in a dense vector space (~few hundred real numbers), where the location and distance between words indicates how similar they are semantically.


<div style="width: 900px; overflow: hidden;">
    <img src="https://developers.google.com/static/machine-learning/guides/text-classification/images/WordEmbeddings.png" width="100%" alt="Your Image">
</div>

Sequence models often have such an embedding layer as their first layer. This layer learns to **turn word index sequences into word embedding vectors** during the training process, such that **each word index gets mapped to a dense vector** of real values representing that word’s location in semantic space.

<div style="width: 900px; overflow: hidden;">
    <img src="https://developers.google.com/static/machine-learning/guides/text-classification/images/EmbeddingLayer.png" width="100%" alt="Your Image">
</div>

<br>

**3. Feature selection**

As in n-gram models, not all words in our data contribute to label predictions. We can optimize our learning process by discarding rare or irrelevant words from our vocabulary. Again, using the most frequent 20,000 features is generally sufficient.

<br>

The following code:

- Tokenizes the texts into words.
- Creates a vocabulary using the top 20,000 tokens.
- Converts the tokens into sequence vectors.
- Pads the sequences to a fixed sequence length.

Vectorization parameters:

In [51]:
# Limit on the number of features. We use the top 20K features
TOP_K = 20000

# Limit on the length of text sequences (sequences longer than this will be truncated)
MAX_SEQUENCE_LENGTH = 500

In [53]:
def sequence_vectorize(train_texts, val_texts):
    """Vectorizes texts as sequence vectors

    1 text = 1 sequence vector with fixed length

    Arguments
    - train_texts: list, training text strings
    - val_texts: list, validation text strings

    # Returns
    - x_train, x_val, word_index: vectorized training and validation texts and word index dictionary.
    """
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index

## Build Models

### Output layer

When there are **only 2 classes (binary classification)**, the model should **output a single probability score**. For instance, outputting 0.14 for a given input sample means “14% confidence that this sample is in class 1, 86% that it is in class 2.” To output such a probability score, the activation function of the last layer should be a sigmoid function, and the loss function used to train the model should be binary cross-entropy.

When there are **more than 2 classes (multi-class classification)**, the model should **output one probability score per class**. The sum of these scores should be 1. For instance, outputting {0: 0.01, 1: 0.14, 2: 0.85} means “1% confidence that this sample is in class 1, 14% that it is in class 2, and 85% that it is in class 3.” To output these scores, the activation function of the last layer should be softmax, and the loss function used to train the model should be categorical cross-entropy.


<div style="width: 900px; overflow: hidden;">
    <img src="https://developers.google.com/static/machine-learning/guides/text-classification/images/LastLayer.png" width="100%" alt="Your Image">
</div>

The following code defines a function that takes the number of classes as input, and outputs the appropriate output layer **# of units** (1 unit for binary classification; otherwise 1 unit for each class) and **activation function**:

In [ ]:
def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.

    Arguments
    - num_classes: int, number of classes.

    Returns
    - units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

### N-gram model

**Multi-layer perceptrons (MLPs)** typically perform better since they are simple to define and understand, provide good accuracy, and require relatively little computation. The following code defines a MLP model using tensorflow.keras, adding Dropout layers for regularization (to prevent overfitting to training samples):

In [1]:
def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    Arguments
    - layers: int, number of `Dense` layers in the model.
    - units: int, output dimension of the layers.
    - dropout_rate: float, percentage of input to drop at Dropout layers.
    - input_shape: tuple, shape of input to the model.
    - num_classes: int, number of output classes.

    Returns
    - An MLP model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    return model

### Sequence model

As mentioned before, sequence models that can **learn from the adjacency of tokens**. They generally have a larger number of parameters to learn. The first layer in these models is an **embedding layer**, which learns the relationship between the words in a dense vector space. Learning word relationships works best over many samples.

Words in a given dataset are most likely not unique to that dataset. We can thus learn the relationship between the words in our dataset using other datasets. To do so, we can transfer an embedding learned from another dataset into our embedding layer. These embeddings are referred to as **pre-trained embeddings**. Using a pre-trained embedding gives the model a head start in the learning process.

There are pre-trained embeddings available that have been trained using large corpora, such as [GloVe](https://nlp.stanford.edu/projects/glove/). GloVe embeddings trained on Wikipedia data may not align with the language patterns in our IMDb dataset. The relationships inferred may need some updating (the embedding weights may need contextual tuning). We can do this in two stages:

- In the first run, with the embedding layer weights frozen, we allow the rest of the network to learn. At the end of this run, the model weights reach a state that is much better than their uninitialized values. For the second run, we allow the embedding layer to also learn, making fine adjustments to all weights in the network. We refer to this process as using a **fine-tuned embedding**.

- Fine-tuned embeddings yield better accuracy. However, this comes at the expense of increased compute power required to train the network. Given a sufficient number of samples (the ratio of *number of samples* to *number of words per sample* > 15k), we could do just as well **learning an embedding from scratch**.

**sepCNNs**, a convolutional network variant, is often more efficient and perform better than other models.

In [ ]:
def sepcnn_model(blocks, filters, kernel_size, embedding_dim, dropout_rate, pool_size,
                 input_shape, num_classes, num_features, use_pretrained_embedding=False,
                 is_embedding_trainable=False, embedding_matrix=None):
    """Creates an instance of a separable CNN model.

    Arguments
    - blocks: int, number of pairs of sepCNN and pooling blocks in the model.
    - filters: int, output dimension of the layers.
    - kernel_size: int, length of the convolution window.
    - embedding_dim: int, dimension of the embedding vectors.
    - dropout_rate: float, percentage of input to drop at Dropout layers.
    - pool_size: int, factor by which to downscale input at MaxPooling layer.
    - input_shape: tuple, shape of input to the model.
    - num_classes: int, number of output classes.
    - num_features: int, number of words (embedding input dimension).
    - use_pretrained_embedding: bool, true if pre-trained embedding is on.
    - is_embedding_trainable: bool, true if embedding layer is trainable.
    - embedding_matrix: dict, dictionary with embedding coefficients.

    Returns
    - A sepCNN model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()

    # Add embedding layer. If pre-trained embedding is used add weights to the
    # embeddings layer and set trainable to input is_embedding_trainable flag.
    if use_pretrained_embedding:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0],
                            weights=[embedding_matrix],
                            trainable=is_embedding_trainable))
    else:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0]))

    for _ in range(blocks-1):
        model.add(Dropout(rate=dropout_rate))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(MaxPooling1D(pool_size=pool_size))

    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(op_units, activation=op_activation))
    return model